In [1]:
import pandas as pd
from collections import defaultdict
infile="/Users/irffanalahi/Research/Research_update/SM/ShowcaseSM/BL17_14/finalref/BL14_othersmrelated/BL14in_out_mincpg2_q0.5_addedcol_q1e-05_maxCompartmentwisedelta-0.7_top-1_singleCpG_final_record.txt"

fragsize=170


indf=pd.read_csv(infile,sep="\t")
indf.head()

,DMRchr,DMRstart,DMRend,chrom,start,end,celltype
0,chr14,92990527,92990670,chr14,92990526,92990528,[DC]
1,chr14,92990527,92990670,chr14,92990556,92990558,[DC]
2,chr14,92990527,92990670,chr14,92990659,92990661,[DC]
3,chr14,92990527,92990670,chr14,92990668,92990670,[DC]
4,chr21,43691129,43691523,chr21,43691128,43691130,[DC]


In [2]:
celllist=list(set((indf['celltype']).tolist()))
print(celllist)

['[DC]', '[nB]', '[mNeu]', '[Er]', '[Eo]', '[mB]', '[NK]']


In [3]:
outdict=defaultdict(list)


grouped=indf.groupby(['DMRchr','DMRstart','DMRend'])

for name, group in grouped:
    outdict['chrom'].append(name[0])
    outdict['start'].append(name[1])
    outdict['end'].append(name[2])
    dlength=int(name[2])-int(name[1])
    outdict['DMRlength'].append(dlength)
    reporter=1.0*dlength/fragsize  ################
    outdict['reporter'].append(reporter)
    
    outdict['#cpg'].append(group.shape[0])
    outdict['celltype'].append((group['celltype'].tolist())[0])

    
    
outdf=pd.DataFrame.from_dict(outdict)

#outdf['cpg_per_reporter']=outdf['reporter']/outdf['#cpg']

outdf.head()

,chrom,start,end,DMRlength,reporter,#cpg,celltype
0,chr1,1221864,1222863,999,5.876471,36,[NK]
1,chr1,3664756,3665237,481,2.829412,13,[mB]
2,chr1,6282776,6283095,319,1.876471,8,[Eo]
3,chr1,10376568,10376667,99,0.582353,7,[Eo]
4,chr1,10381038,10382014,976,5.741176,13,[Eo]


In [4]:
summarydict=defaultdict(list)
outdfgr=outdf.groupby(['celltype'])
for name, group in outdfgr:
    summarydict['celltype'].append(name)
    summarydict['total_DMR'].append(group.shape[0])
    summarydict['avg_cpg_per_DMR'].append(group['#cpg'].mean())
    summarydict['total_reporter'].append(group['reporter'].sum())

summarydf=pd.DataFrame.from_dict(summarydict)

summarydf.head()
    

,celltype,total_DMR,avg_cpg_per_DMR,total_reporter
0,[DC],4,6.750000,6.405882
1,[Eo],61,8.983607,107.176471
2,[Er],5,7.800000,6.505882
3,[NK],133,8.962406,216.029412
4,[mB],170,9.823529,339.852941


In [5]:
outname=infile+"_stats.txt"
outdf.to_csv(outname,sep="\t",index=False)
summarydf.to_csv(outname+"_summary.txt",sep="\t",index=False)